<a href="https://colab.research.google.com/github/akinahomwabella/Deep_Fake_Detection/blob/main/Deepfake_Detection_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
!pip install --upgrade kagglehub
import kagglehub
print(kagglehub.__version__)


0.3.3


In [ ]:
import os

# Step 1: Create the directory 'deepfake-detection-challenge' in your Google Drive if it doesn't exist
os.makedirs('/content/drive/MyDrive/deepfake-detection-challenge/', exist_ok=True)

# Step 2: Change the working directory to 'deepfake-detection-challenge'
%cd '/content/drive/MyDrive/deepfake-detection-challenge/'

# Step 3: Install the Kaggle API client if you don't have it
!pip install kaggle
from google.colab import files
files.upload()
# Step 4: Make a directory for the Kaggle API key and move your Kaggle API credentials (kaggle.json)
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Step 5: Download the dataset (adjust the competition or dataset slug accordingly)
!kaggle competitions download -c deepfake-detection-challenge

# Step 6: Unzip the downloaded dataset into the 'deepfake-detection-challenge' directory
!unzip deepfake-detection-challenge.zip -d /content/drive/MyDrive/deepfake-detection-challenge/


/content/drive/MyDrive/deepfake-detection-challenge


Saving kaggle.json to kaggle (12).json
deepfake-detection-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  deepfake-detection-challenge.zip
replace /content/drive/MyDrive/deepfake-detection-challenge/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/drive/MyDrive/deepfake-detection-challenge/sample_submission.csv  
  inflating: /content/drive/MyDrive/deepfake-detection-challenge/test_videos/aassnaulhq.mp4  
  inflating: /content/drive/MyDrive/deepfake-detection-challenge/test_videos/aayfryxljh.mp4  
  inflating: /content/drive/MyDrive/deepfake-detection-challenge/test_videos/acazlolrpz.mp4  
  inflating: /content/drive/MyDrive/deepfake-detection-challenge/test_videos/adohdulfwb.mp4  
  inflating: /content/drive/MyDrive/deepfake-detection-challenge/test_videos/ahjnxtiamx.mp4  
  inflating: /content/drive/MyDrive/deepfake-detection-challenge/test_videos/ajiyrjfyzp.mp4  
  inflating: /content/driv

In [ ]:
#Downloading the Forensic dataset
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# Create the directory 'faceforensics_data' in your Google Drive if it doesn't exist
os.makedirs('/content/drive/MyDrive/faceforensics_data', exist_ok=True)

# Change the working directory to 'faceforensics_data'
%cd /content/drive/MyDrive/faceforensics_data/


/content/drive/MyDrive/faceforensics_data


In [ ]:
!pip install tqdm mtcnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.2 MB/s eta 0:00:00


In [ ]:
#Downloaded the Face Forensics to Google Colab
import os
import urllib
import urllib.request
import tempfile
import time
import sys
import json
import random
from tqdm import tqdm
from os.path import join

# URLs and filenames
FILELIST_URL = 'misc/filelist.json'
DEEPFEAKES_DETECTION_URL = 'misc/deepfake_detection_filenames.json'
DEEPFAKES_MODEL_NAMES = ['decoder_A.h5', 'decoder_B.h5', 'encoder.h5',]

# Parameters
DATASETS = {
    'original_youtube_videos': 'misc/downloaded_youtube_videos.zip',
    'original_youtube_videos_info': 'misc/downloaded_youtube_videos_info.zip',
    'original': 'original_sequences/youtube',
    'DeepFakeDetection_original': 'original_sequences/actors',
    'Deepfakes': 'manipulated_sequences/Deepfakes',
    'DeepFakeDetection': 'manipulated_sequences/DeepFakeDetection',
    'Face2Face': 'manipulated_sequences/Face2Face',
    'FaceShifter': 'manipulated_sequences/FaceShifter',
    'FaceSwap': 'manipulated_sequences/FaceSwap',
    'NeuralTextures': 'manipulated_sequences/NeuralTextures'
    }
ALL_DATASETS = ['original', 'DeepFakeDetection_original', 'Deepfakes',
                'DeepFakeDetection', 'Face2Face', 'FaceShifter', 'FaceSwap',
                'NeuralTextures']
COMPRESSION = ['raw', 'c23', 'c40']
TYPE = ['videos', 'masks', 'models']
SERVERS = ['EU', 'EU2', 'CA']

def parse_args():
    class Args:
        def __init__(self):
            self.output_path ='/content/drive/My Drive/faceforensics_data' #if using Google Drive
            self.dataset = 'Deepfakes'  # Choose from DATASETS keys or 'all'
            self.compression = 'c23'  # Choose from ['raw', 'c23', 'c40']
            self.type = 'videos'  # Choose from ['videos', 'masks', 'models']
            self.num_videos = 50  # Set to an integer or None for all videos
            self.server = 'EU2'  # Choose from ['EU', 'EU2', 'CA']
    args = Args()
    # URLs
    server = args.server
    if server == 'EU':
        server_url = 'http://canis.vc.in.tum.de:8100/'
    elif server == 'EU2':
        server_url = 'http://kaldir.vc.in.tum.de/faceforensics/'
    elif server == 'CA':
        server_url = 'http://falas.cmpt.sfu.ca:8100/'
    else:
        raise Exception('Wrong server name. Choices: {}'.format(str(SERVERS)))
    args.tos_url = server_url + 'webpage/FaceForensics_TOS.pdf'
    args.base_url = server_url + 'v3/'
    args.deepfakes_model_url = server_url + 'v3/manipulated_sequences/' + \
                               'Deepfakes/models/'
    return args

def download_files(filenames, base_url, output_path, report_progress=True):
    os.makedirs(output_path, exist_ok=True)
    if report_progress:
        filenames = tqdm(filenames)
    for filename in filenames:
        download_file(base_url + filename, join(output_path, filename))

def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    if duration == 0:
        duration = 1e-6  # avoid division by zero
    progress_size = int(count * block_size)
    speed = int(progress_size / (1024 * duration))
    percent = int(min(100, count * block_size * 100 / total_size))
    sys.stdout.write("\rProgress: %d%%, %d MB, %d KB/s, %d seconds passed" %
                     (percent, progress_size / (1024 * 1024), speed, duration))
    sys.stdout.flush()

def download_file(url, out_file, report_progress=False):
    out_dir = os.path.dirname(out_file)
    os.makedirs(out_dir, exist_ok=True)
    if not os.path.isfile(out_file):
        fh, out_file_tmp = tempfile.mkstemp(dir=out_dir)
        f = os.fdopen(fh, 'w')
        f.close()
        try:
            if report_progress:
                urllib.request.urlretrieve(url, out_file_tmp,
                                           reporthook=reporthook)
            else:
                urllib.request.urlretrieve(url, out_file_tmp)
            os.rename(out_file_tmp, out_file)
        except Exception as e:
            print(f"Error downloading {url}: {e}")
            if os.path.exists(out_file_tmp):
                os.remove(out_file_tmp)
    else:
        tqdm.write('WARNING: skipping download of existing file ' + out_file)

def main(args):
    # TOS
    print('By running this script, you confirm that you have agreed to the FaceForensics terms of use as described at:')
    print(args.tos_url)
    print('***')
    # Proceeding without input

    # Extract arguments
    c_datasets = [args.dataset] if args.dataset != 'all' else ALL_DATASETS
    c_type = args.type
    c_compression = args.compression
    num_videos = args.num_videos
    output_path = args.output_path
    os.makedirs(output_path, exist_ok=True)

    # Check for special dataset cases
    for dataset in c_datasets:
        dataset_path = DATASETS[dataset]
        # Special cases
        if 'original_youtube_videos' in dataset:
            # Here we download the original youtube videos zip file
            print('Downloading original youtube videos.')
            if not 'info' in dataset_path:
                print('Please be patient, this may take a while (~40gb)')
                suffix = ''
            else:
                suffix = 'info'
            download_file(args.base_url + '/' + dataset_path,
                          out_file=join(output_path,
                                        'downloaded_videos{}.zip'.format(
                                            suffix)),
                          report_progress=True)
            return

        # Else: regular datasets
        print('Downloading {} of dataset "{}"'.format(
            c_type, dataset_path
        ))

        # Get filelists and video lengths list from server
        if 'DeepFakeDetection' in dataset_path or 'actors' in dataset_path:
            filepaths = json.loads(urllib.request.urlopen(args.base_url + '/' +
                DEEPFEAKES_DETECTION_URL).read().decode("utf-8"))
            if 'actors' in dataset_path:
                filelist = filepaths['actors']
            else:
                filelist = filepaths['DeepFakesDetection']
        elif 'original' in dataset_path:
            # Load filelist from server
            file_pairs = json.loads(urllib.request.urlopen(args.base_url + '/' +
                FILELIST_URL).read().decode("utf-8"))
            filelist = []
            for pair in file_pairs:
                filelist += pair
        else:
            # Load filelist from server
            file_pairs = json.loads(urllib.request.urlopen(args.base_url + '/' +
                FILELIST_URL).read().decode("utf-8"))
            # Get filelist
            filelist = []
            for pair in file_pairs:
                filelist.append('_'.join(pair))
                if c_type != 'models':
                    filelist.append('_'.join(pair[::-1]))
        # Maybe limit number of videos for download
        if num_videos is not None and num_videos > 0:
            print('Downloading the first {} videos'.format(num_videos))
            filelist = filelist[:num_videos]

        # Server and local paths
        dataset_videos_url = args.base_url + '{}/{}/{}/'.format(
            dataset_path, c_compression, c_type)
        dataset_mask_url = args.base_url + '{}/{}/videos/'.format(
            dataset_path, 'masks', c_type)

        if c_type == 'videos':
            dataset_output_path = join(output_path, dataset_path, c_compression,
                                       c_type)
            print('Output path: {}'.format(dataset_output_path))
            filelist = [filename + '.mp4' for filename in filelist]
            download_files(filelist, dataset_videos_url, dataset_output_path)
        elif c_type == 'masks':
            dataset_output_path = join(output_path, dataset_path, c_type,
                                       'videos')
            print('Output path: {}'.format(dataset_output_path))
            if 'original' in dataset:
                if args.dataset != 'all':
                    print('Only videos available for original data. Aborting.')
                    return
                else:
                    print('Only videos available for original data. '
                          'Skipping original.\n')
                    continue
            if 'FaceShifter' in dataset:
                print('Masks not available for FaceShifter. Aborting.')
                return
            filelist = [filename + '.mp4' for filename in filelist]
            download_files(filelist, dataset_mask_url, dataset_output_path)

        # Else: models for Deepfakes
        else:
            if dataset != 'Deepfakes' and c_type == 'models':
                print('Models only available for Deepfakes. Aborting')
                return
            dataset_output_path = join(output_path, dataset_path, c_type)
            print('Output path: {}'.format(dataset_output_path))

            # Get Deepfakes models
            for folder in tqdm(filelist):
                folder_filelist = DEEPFAKES_MODEL_NAMES

                # Folder paths
                folder_base_url = args.deepfakes_model_url + folder + '/'
                folder_dataset_output_path = join(dataset_output_path,
                                                  folder)
                download_files(folder_filelist, folder_base_url,
                               folder_dataset_output_path,
                               report_progress=False)   # already done

# Run the script
args = parse_args()
main(args)


By running this script, you confirm that you have agreed to the FaceForensics terms of use as described at:
http://kaldir.vc.in.tum.de/faceforensics/webpage/FaceForensics_TOS.pdf
***
Output path: /content/drive/My Drive/faceforensics_data/manipulated_sequences/Deepfakes/c23/videos


 64%|██████▍   | 32/50 [02:44<00:53,  2.96s/it]

 64%|██████▍   | 32/50 [02:44<00:53,  2.96s/it]

100%|██████████| 50/50 [02:44<00:00,  3.30s/it]

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
!pip install tensorflow==2.17.0 tf-keras==2.17.0 typing-extensions==4.10.0


  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openai 1.52.2 requires typing-extensions<5,>=4.11, but you have typing-extensions 4.10.0 which is incompatible.


In [ ]:
import torch
torch.cuda.is_available()

False

In [ ]:
!pip install mtcnn opencv-python dlib


In [ ]:
#Uploaded the Celeb-DF-2 file
import os
folder_path = '/content/drive/My Drive/Celeb-DF-v2'

# List all files in the folder
files = os.listdir(folder_path)
print(files)

['List_of_testing_videos.txt', 'Celeb-synthesis', 'YouTube-real', 'Celeb-real']


In [ ]:
!pip install mtcnn
!pip install Pillow

In [ ]:
# Download the file using wget
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
# Extract the .bz2 file
!bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2
# Verify the file is in the current directory
!ls

--2024-11-07 00:56:09--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2.21’

shape_predictor_68_ 100%[===================>]  61.07M  39.2MB/s    in 1.6s    

2024-11-07 00:56:11 (39.2 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2.21’ saved [64040097/64040097]

bzip2: Output file shape_predictor_68_face_landmarks.dat already exists.
cnn_lstm_final_model.keras		      shape_predictor_68_face_landmarks.dat.bz2.18
labels.csv				      shape_predictor_68_face_landmarks.dat.bz2.19
manipulated_sequences			      shape_predictor_68_face_landmarks.dat.bz2.2
shape_predictor_68_face_landmarks.dat	      shape_predictor_68_face_landmarks.dat.bz2.20
shape_predictor_68_face_landmarks.dat.bz2     shape_predictor_68_face_landmarks.dat.bz2.21
shape_pre

In [ ]:
import json

# Path to the metadata file (adjust this based on your file location)
metadata_path = '/content/drive/MyDrive/deepfake-detection-challenge/train_sample_videos/metadata.json'

# Open and load the metadata file
with open(metadata_path, 'r') as f:
    metadata = json.load(f)

# Inspect the metadata content (it will contain video filenames and labels)
print(json.dumps(metadata, indent=4))


{
    "aagfhgtpmv.mp4": {
        "label": "FAKE",
        "split": "train",
        "original": "vudstovrck.mp4"
    },
    "aapnvogymq.mp4": {
        "label": "FAKE",
        "split": "train",
        "original": "jdubbvfswz.mp4"
    },
    "abarnvbtwb.mp4": {
        "label": "REAL",
        "split": "train",
        "original": null
    },
    "abofeumbvv.mp4": {
        "label": "FAKE",
        "split": "train",
        "original": "atvmxvwyns.mp4"
    },
    "abqwwspghj.mp4": {
        "label": "FAKE",
        "split": "train",
        "original": "qzimuostzz.mp4"
    },
    "acifjvzvpm.mp4": {
        "label": "FAKE",
        "split": "train",
        "original": "kbvibjhfzo.mp4"
    },
    "acqfdwsrhi.mp4": {
        "label": "FAKE",
        "split": "train",
        "original": "ccfoszqabv.mp4"
    },
    "acxnxvbsxk.mp4": {
        "label": "FAKE",
        "split": "train",
        "original": "fjlyaizcwc.mp4"
    },
    "acxwigylke.mp4": {
        "label": "FAKE",
        "

In [ ]:
import os
import csv
import json

# Dataset paths for real and fake files
celeb_real_paths = ['/content/drive/My Drive/Celeb-DF-v2/Celeb-real/', '/content/drive/My Drive/Celeb-DF-v2/YouTube-real/']
celeb_fake_path = ['/content/drive/My Drive/Celeb-DF-v2/Celeb-synthesis/']
forensic_path_fake = ['/content/drive/My Drive/faceforensics_data/manipulated_sequences']
forensic_path_real = ['/content/drive/My Drive/faceforensics_data/original_sequences']
dfdc_real_paths = []
dfdc_fake_paths = []

# Path to the Kaggle dataset and metadata file
kaggle_dataset_path = '/content/drive/MyDrive/deepfake-detection-challenge/train_sample_videos/'
metadata_path = os.path.join(kaggle_dataset_path, 'metadata.json')

# Function to collect files from the given paths
def collect_files(paths, label):
    file_list = []
    for path in paths:
        for root, dirs, files in os.walk(path):
            for file in files:
                if file.endswith(('.jpg', '.jpeg', '.png', '.mp4')):
                    file_list.append((os.path.join(root, file), label))
    return file_list

# Load the metadata from the Kaggle dataset
with open(metadata_path, 'r') as f:
    metadata = json.load(f)

# Collect real and fake video files based on metadata from Kaggle dataset
for video_name, data in metadata.items():
    video_path = os.path.join(kaggle_dataset_path, video_name)
    if data['label'] == 'REAL':
        dfdc_real_paths.append((video_path, 0))
    elif data['label'] == 'FAKE':
        dfdc_fake_paths.append((video_path, 1))

# Collect files from Celeb-DF and FaceForensics datasets
real_files = collect_files(celeb_real_paths + forensic_path_real, 0)
fake_files = collect_files(celeb_fake_path + forensic_path_fake, 1)

# Append the Kaggle dataset (DFDC) real and fake files to the lists
real_files.extend(dfdc_real_paths)
fake_files.extend(dfdc_fake_paths)

# Ensure the directory 'labeled_data' exists
os.makedirs('/content/drive/My Drive/labeled_data', exist_ok=True)

# Now, write the CSV file in the 'labeled_data' directory
csv_file_path = '/content/drive/My Drive/labeled_data/labeled_data.csv'
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File', 'Label'])  # Write header
    writer.writerows(real_files + fake_files)  # Write the file paths and labels

print(f"Labeling complete and saved to '{csv_file_path}'")

# Read and print the contents of the CSV file
with open(csv_file_path, 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        print(row)  # Print each row of the CSV file


Streaming output truncated to the last 5000 lines.
['/content/drive/My Drive/faceforensics_data/manipulated_sequences/Deepfakes/c23/videos/339_392_bright_338.jpg', '1']
['/content/drive/My Drive/faceforensics_data/manipulated_sequences/Deepfakes/c23/videos/339_392_flipped_339.jpg', '1']
['/content/drive/My Drive/faceforensics_data/manipulated_sequences/Deepfakes/c23/videos/339_392_rotated_339.jpg', '1']
['/content/drive/My Drive/faceforensics_data/manipulated_sequences/Deepfakes/c23/videos/339_392_bright_339.jpg', '1']
['/content/drive/My Drive/faceforensics_data/manipulated_sequences/Deepfakes/c23/videos/339_392_flipped_340.jpg', '1']
['/content/drive/My Drive/faceforensics_data/manipulated_sequences/Deepfakes/c23/videos/339_392_rotated_340.jpg', '1']
['/content/drive/My Drive/faceforensics_data/manipulated_sequences/Deepfakes/c23/videos/339_392_bright_340.jpg', '1']
['/content/drive/My Drive/faceforensics_data/manipulated_sequences/Deepfakes/c23/videos/339_392_flipped_341.jpg', '1']


In [ ]:
# Download the file using wget
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
# Extract the .bz2 file
!bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2

--2024-11-07 00:58:26--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2.22’

shape_predictor_68_ 100%[===================>]  61.07M  11.4MB/s    in 5.4s    

2024-11-07 00:58:32 (11.4 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2.22’ saved [64040097/64040097]

bzip2: Output file shape_predictor_68_face_landmarks.dat already exists.


In [ ]:
!pip install --upgrade numpy==1.25.2
!pip install --upgrade pandas
!pip install --upgrade mtcnn
from mtcnn.mtcnn import MTCNN
import os
import cv2
import random
import dlib
import csv

# Load Dlib's face landmark predictor
predictor_path = 'shape_predictor_68_face_landmarks.dat'  # Ensure this file is downloaded
detector = MTCNN()
predictor = dlib.shape_predictor(predictor_path)
face_detector_dlib = dlib.get_frontal_face_detector()

# Path to the CSV file containing file paths and labels
csv_file_path = '/content/drive/My Drive/labeled_data/labeled_data.csv'

# Directory to save the images with landmarks
output_dir = '/content/drive/MyDrive/landmarked_images/'
os.makedirs(output_dir, exist_ok=True)

# Set the limit for the number of images to process per dataset
image_limit_per_dataset = 100

# Read the CSV file and process each image or video
with open(csv_file_path, 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header
    file_paths = [row[0] for row in reader]  # Collect file paths from the CSV

# Shuffle and select a random subset of images for the current dataset
random.shuffle(file_paths)
selected_image_files = file_paths[:image_limit_per_dataset]

# Process each image or video
for img_path in selected_image_files:
    image = cv2.imread(img_path)
    if image is not None:
        # Resize image for faster detection
        image_resized = cv2.resize(image, (500, 500))

        # Detect faces using MTCNN
        faces = detector.detect_faces(image_resized)

        for i, face in enumerate(faces):
            x, y, w, h = face['box']

            # Resize the coordinates to the original image size
            factor_x = image.shape[1] / image_resized.shape[1]
            factor_y = image.shape[0] / image_resized.shape[0]
            x = int(x * factor_x)
            y = int(y * factor_y)
            w = int(w * factor_x)
            h = int(h * factor_y)

            # Crop the face
            face_img = image[y:y+h, x:x+w]

            # Detect landmarks with Dlib
            dlib_rect = dlib.rectangle(x, y, x + w, y + h)
            landmarks = predictor(image, dlib_rect)

            # Extract specific prominent features (eyes, nose, mouth)
            landmarks_points = []
            for n in range(0, 68):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
                landmarks_points.append((x, y))

            # Draw circles on eyes, mouth, and nose
            left_eye_points = landmarks_points[36:42]  # Left eye
            right_eye_points = landmarks_points[42:48]  # Right eye
            nose_points = landmarks_points[27:36]  # Nose
            mouth_points = landmarks_points[48:61]  # Mouth

            for point in left_eye_points + right_eye_points + nose_points + mouth_points:
                cv2.circle(face_img, (point[0], point[1]), 2, (0, 255, 0), -1)

            # Save the image with landmarks
            face_filename = f"{os.path.splitext(os.path.basename(img_path))[0]}_landmarks.jpg"
            face_path = os.path.join(output_dir, face_filename)
            cv2.imwrite(face_path, face_img)
            print(f"Saved image with landmarks to {face_path}")
    else:
        print(f"Failed to load image from {img_path}")




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 8.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 71.4 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [ ]:
import cv2
import os
import csv
import random
import numpy as np

# Define augmentation functions
def drop_frames(frames, drop_rate=0.2):
    """Randomly drops frames from the input list based on the drop_rate."""
    return [frame for i, frame in enumerate(frames) if np.random.rand() > drop_rate]

def time_shift_frames(frames, shift_rate=2):
    """Performs time shifting on the frames by shifting them by shift_rate."""
    return frames[shift_rate:] + frames[:shift_rate]

def augment_image(image):
    """Applies flipping, rotating, and brightness adjustment to the input image."""
    # Flip the image horizontally
    flipped_image = cv2.flip(image, 1)

    # Rotate the image by 90 degrees
    height, width = image.shape[:2]
    center = (width // 2, height // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, 90, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))

    # Brightness adjustment
    brightness_adjusted = cv2.convertScaleAbs(image, alpha=1.5, beta=0)

    return flipped_image, rotated_image, brightness_adjusted

def save_video(frames, output_path, fps, frame_size):
    """Save frames as a video."""
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 files
    out = cv2.VideoWriter(output_path, fourcc, fps, frame_size)

    for frame in frames:
        out.write(frame)

    out.release()

# Path to the CSV file containing file paths and labels (real or fake)
csv_file_path = '/content/drive/My Drive/labeled_data/labeled_data.csv'

# Directory to save the augmented videos
augmented_dir = '/content/drive/MyDrive/augmented_videos/'
os.makedirs(augmented_dir, exist_ok=True)

# Set the limit for the number of videos to process
video_limit = 50

# Read the CSV file and process each video
with open(csv_file_path, 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header
    video_paths = [row[0] for row in reader]  # Collect file paths from the CSV

# Shuffle and select a random subset of videos for augmentation
random.shuffle(video_paths)
selected_video_files = video_paths[:video_limit]

# Process each video
for video_path in selected_video_files:
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Failed to open video {video_path}")
        continue

    frames = []
    frame_idx = 0
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    # Read frames from video
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
        frame_idx += 1

    # Apply frame dropping and time shifting
    frames = drop_frames(frames)
    frames = time_shift_frames(frames)

    # Prepare augmented video frames
    flipped_frames = []
    rotated_frames = []
    brightness_adjusted_frames = []

    # Apply augmentation to each frame
    for idx, frame in enumerate(frames):
        flipped, rotated, brightness = augment_image(frame)
        flipped_frames.append(flipped)
        rotated_frames.append(rotated)
        brightness_adjusted_frames.append(brightness)

    # Save augmented videos (flipped, rotated, brightness adjusted)
    base_name = os.path.splitext(os.path.basename(video_path))[0]

    flipped_output_path = os.path.join(augmented_dir, f"{base_name}_flipped.mp4")
    rotated_output_path = os.path.join(augmented_dir, f"{base_name}_rotated.mp4")
    brightness_output_path = os.path.join(augmented_dir, f"{base_name}_bright.mp4")

    # Save the flipped video
    save_video(flipped_frames, flipped_output_path, fps, frame_size)
    print(f"Saved flipped video to {flipped_output_path}")

    # Save the rotated video
    save_video(rotated_frames, rotated_output_path, fps, frame_size)
    print(f"Saved rotated video to {rotated_output_path}")

    # Save the brightness-adjusted video
    save_video(brightness_adjusted_frames, brightness_output_path, fps, frame_size)
    print(f"Saved brightness-adjusted video to {brightness_output_path}")

    cap.release()


Saved flipped video to /content/drive/MyDrive/augmented_videos/585_599_rotated_262_flipped.mp4
Saved rotated video to /content/drive/MyDrive/augmented_videos/585_599_rotated_262_rotated.mp4
Saved brightness-adjusted video to /content/drive/MyDrive/augmented_videos/585_599_rotated_262_bright.mp4
Saved flipped video to /content/drive/MyDrive/augmented_videos/147_055_rotated_528_flipped.mp4
Saved rotated video to /content/drive/MyDrive/augmented_videos/147_055_rotated_528_rotated.mp4
Saved brightness-adjusted video to /content/drive/MyDrive/augmented_videos/147_055_rotated_528_bright.mp4
Saved flipped video to /content/drive/MyDrive/augmented_videos/657_644_flipped_373_flipped.mp4
Saved rotated video to /content/drive/MyDrive/augmented_videos/657_644_flipped_373_rotated.mp4
Saved brightness-adjusted video to /content/drive/MyDrive/augmented_videos/657_644_flipped_373_bright.mp4
Saved flipped video to /content/drive/MyDrive/augmented_videos/175_159_bright_161_flipped.mp4
Saved rotated vide

In [ ]:
!pip install --upgrade imageio
!pip install imageio-ffmpeg
!sudo apt-get update && sudo apt-get install ffmpeg -y

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,611 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,241 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages 

In [ ]:
import os
import cv2
import numpy as np
import logging
import traceback
import time
import subprocess
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input, LSTM, TimeDistributed, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from collections import Counter
import tensorflow as tf

# Check versions
print("OpenCV version:", cv2.__version__)
print("NumPy version:", np.__version__)
print("TensorFlow version:", tf.__version__)

# Check GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Set up logging
logging.basicConfig(level=logging.DEBUG,
                    filename='ffmpeg_debug.log',
                    filemode='w',
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Function to convert videos to a standard format
def convert_video_format(filepath):
    """Convert video to MP4 format with H.264 encoding using FFmpeg."""
    try:
        new_filepath = filepath + '_converted.mp4'
        command = [
            'ffmpeg',
            '-y',  # Overwrite output files without asking
            '-i', filepath,
            '-c:v', 'libx264',
            '-crf', '23',
            '-preset', 'medium',
            '-c:a', 'aac',
            '-strict', 'experimental',
            new_filepath
        ]
        subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if os.path.exists(new_filepath):
            return new_filepath
        else:
            logging.error(f"Failed to convert video: {filepath}")
            return None
    except Exception as e:
        logging.error(f"Error converting video {filepath}: {e}", exc_info=True)
        return Nonevv

# Function to load and preprocess images or videos
def load_and_preprocess_file(filepath, frame_skip=2, max_frames=16, sequence_length=16):
    """Load and preprocess image or video files."""
    try:
        if not os.path.exists(filepath):
            logging.error(f"File does not exist: {filepath}")
            return None

        if not os.access(filepath, os.R_OK):
            logging.error(f"File cannot be accessed (no read permissions): {filepath}")
            return None

        if filepath.lower().endswith(('.png', '.jpg', '.jpeg')):
            img = load_img(filepath, target_size=(224, 224))
            img = img_to_array(img)
            img = img / 255.0  # Normalize the image
            return [img]

        elif filepath.lower().endswith(('.mp4', '.avi', '.mov')):
            video_frames = []
            cap = cv2.VideoCapture(filepath)
            if not cap.isOpened():
                logging.error(f"Failed to open video with OpenCV: {filepath}")
                # Try to convert video to standard format
                converted_filepath = convert_video_format(filepath)
                if converted_filepath:
                    cap = cv2.VideoCapture(converted_filepath)
                    if not cap.isOpened():
                        logging.error(f"Failed to open video after conversion: {converted_filepath}")
                        return None
                    else:
                        logging.info(f"Video converted successfully: {converted_filepath}")
                else:
                    return None

            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            if total_frames < sequence_length * frame_skip:
                logging.warning(f"Not enough frames in video: {filepath}")
                cap.release()
                return None

            frame_count = 0
            while len(video_frames) < max_frames and cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    logging.warning(f"Video ended prematurely: {filepath}")
                    break

                if frame_count % frame_skip == 0:
                    frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)
                    frame = frame.astype('float32') / 255.0
                    video_frames.append(frame)

                frame_count += 1
            cap.release()

            # Pad sequences if they are shorter than sequence_length
            if len(video_frames) < sequence_length:
                logging.warning(f"Frames less than sequence length in file: {filepath}")
                while len(video_frames) < sequence_length:
                    video_frames.append(video_frames[-1])

            return video_frames if video_frames else None
        else:
            logging.warning(f"Unsupported file format: {filepath}")
            return None
    except cv2.error as e:
        logging.error(f"OpenCV error while processing {filepath}: {e}", exc_info=True)
        return None
    except Exception as e:
        logging.error(f"Error processing file {filepath}: {e}", exc_info=True)
        return None

# Function to check if a video has sufficient frames
def has_sufficient_frames(video_path, min_frames=16):
    try:
        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cap.release()
        return total_frames >= min_frames
    except Exception as e:
        logging.error(f"Error checking frames for {video_path}: {e}", exc_info=True)
        return False

# Create sequences of images/frames for CNN-LSTM input
def create_image_sequences(file_list, labels, sequence_length=16):
    X_sequences = []
    y_sequences = []

    for file, label in zip(file_list, labels):
        frames = load_and_preprocess_file(file, sequence_length=sequence_length)
        if frames is not None and len(frames) >= sequence_length:
            frames = frames[:sequence_length]  # Ensure the sequence has the correct length
            X_sequences.append(frames)
            y_sequences.append(label)
        else:
            logging.warning(f"Skipping file {file} due to insufficient frames or loading error.")

    if not X_sequences:
        logging.error("No valid sequences found. Returning empty arrays.")
        return np.empty((0, sequence_length, 224, 224, 3)), np.empty((0,))

    X_sequences = np.array(X_sequences)
    y_sequences = np.array(y_sequences)

    return X_sequences, y_sequences

# Function to load file paths and labels from directories
def load_data_from_directories(image_dir, video_dir):
    file_list = []
    labels = []
    start_time = time.time()

    valid_image_extensions = ('.png', '.jpg', '.jpeg')
    valid_video_extensions = ('.mp4', '.avi', '.mov')

    for root, _, files in os.walk(image_dir):
        for file in files:
            if file.lower().endswith(valid_image_extensions):
                filepath = os.path.join(root, file)
                if os.path.exists(filepath) and os.access(filepath, os.R_OK):
                    file_list.append(filepath)
                    labels.append(1)
                    print(f"Processed image file: {file}, time elapsed: {time.time() - start_time:.2f} seconds")
                else:
                    logging.warning(f"Cannot access image file: {filepath}")
            else:
                continue  # Skip unknown file types

    for root, _, files in os.walk(video_dir):
        for file in files:
            if file.lower().endswith(valid_video_extensions):
                filepath = os.path.join(root, file)
                if os.path.exists(filepath) and os.access(filepath, os.R_OK):
                    file_list.append(filepath)
                    labels.append(0)
                    print(f"Processed video file: {file}, time elapsed: {time.time() - start_time:.2f} seconds")
                else:
                    logging.warning(f"Cannot access video file: {filepath}")
            else:
                continue  # Skip unknown file types

    return file_list, labels

# Load the data from directories
output_dir = '/content/drive/MyDrive/landmarked_images/'
augmented_dir = '/content/drive/MyDrive/augmented_videos/'
file_list, labels = load_data_from_directories(output_dir, augmented_dir)

# Check class distribution
print("Class distribution:", Counter(labels))

# Split data
X_train, X_temp, y_train, y_temp = train_test_split(file_list, labels, test_size=0.3, stratify=labels, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

sequence_length = 16  # Adjust based on your data

# Create sequences for CNN-LSTM model
X_train_seq, y_train_seq = create_image_sequences(X_train, y_train, sequence_length=sequence_length)
X_val_seq, y_val_seq = create_image_sequences(X_val, y_val, sequence_length=sequence_length)

# Print input shapes
print("X_train_seq shape:", X_train_seq.shape)
print("y_train_seq shape:", y_train_seq.shape)
print("X_val_seq shape:", X_val_seq.shape)
print("y_val_seq shape:", y_val_seq.shape)

# Define a CNN-LSTM model
def build_cnn_lstm_model(sequence_length=16):
    input_layer = Input(shape=(sequence_length, 224, 224, 3))
    base_cnn = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    base_cnn.trainable = False  # Freeze the base model

    cnn_output = TimeDistributed(base_cnn)(input_layer)
    x = TimeDistributed(GlobalAveragePooling2D())(cnn_output)
    x = LSTM(128, return_sequences=False)(x)
    x = Dropout(0.5)(x)  # Add dropout for regularization
    output_layer = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build the model
model = build_cnn_lstm_model(sequence_length=sequence_length)
model.summary()

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3)

# Train the model
if X_train_seq.size > 0 and y_train_seq.size > 0 and X_val_seq.size > 0 and y_val_seq.size > 0:
    history = model.fit(
        X_train_seq, y_train_seq,
        validation_data=(X_val_seq, y_val_seq),
        epochs=10,
        batch_size=8,  # Adjust batch size based on your system's memory
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

    # Save the model after training using the native Keras format
    model.save('cnn_lstm_final_model.keras')
    print("Model saved successfully.")
else:
    print("Training skipped due to insufficient training data.")

# Remove or comment out post-training processing code to ensure the script terminates
# For example:
# # Process each image or video
# selected_image_files = file_list  # Assuming this is a list of selected files
# for img_path in selected_image_files:
#     try:
#         # Your processing code here
#         pass
#     except Exception as e:
#         print(f"Error processing file {img_path}: {e}")
#         traceback.print_exc()
#         continue  # Skip to the next file in case of an error

print("Script execution completed.")


Streaming output truncated to the last 5000 lines.
Processed image file: 134_192_bright_230_landmarks.jpg, time elapsed: 1.34 seconds
Processed image file: 339_392_bright_248_landmarks.jpg, time elapsed: 1.34 seconds
Processed image file: 942_943_flipped_308_landmarks.jpg, time elapsed: 1.34 seconds
Processed image file: 992_980_flipped_472_landmarks.jpg, time elapsed: 1.34 seconds
Processed image file: 648_654_flipped_8_landmarks.jpg, time elapsed: 1.34 seconds
Processed image file: id17_0006_bright_50_landmarks.jpg, time elapsed: 1.34 seconds
Processed image file: 210_241_bright_400_landmarks.jpg, time elapsed: 1.34 seconds
Processed image file: 266_252_bright_307_landmarks.jpg, time elapsed: 1.34 seconds
Processed image file: 469_481_flipped_289_landmarks.jpg, time elapsed: 1.34 seconds
Processed image file: 920_811_bright_248_landmarks.jpg, time elapsed: 1.34 seconds
Processed image file: 992_980_rotated_320_landmarks.jpg, time elapsed: 1.34 seconds
Processed image file: 175_159_br

Streaming output truncated to the last 5000 lines.
ERROR:root:Failed to open video with OpenCV: /content/drive/MyDrive/augmented_videos/id13_0001_rotated_42_flipped.mp4
ERROR:root:Failed to convert video: /content/drive/MyDrive/augmented_videos/id13_0001_rotated_42_flipped.mp4
ERROR:root:Failed to open video with OpenCV: /content/drive/MyDrive/augmented_videos/716_712_frame_7_flipped.mp4
ERROR:root:Failed to convert video: /content/drive/MyDrive/augmented_videos/716_712_frame_7_flipped.mp4
ERROR:root:Failed to open video with OpenCV: /content/drive/MyDrive/augmented_videos/284_263_rotated_359_rotated.mp4
ERROR:root:Failed to convert video: /content/drive/MyDrive/augmented_videos/284_263_rotated_359_rotated.mp4
ERROR:root:Failed to open video with OpenCV: /content/drive/MyDrive/augmented_videos/id13_0001_rotated_172_flipped.mp4
ERROR:root:Failed to convert video: /content/drive/MyDrive/augmented_videos/id13_0001_rotated_172_flipped.mp4
ERROR:root:Failed to open video with OpenCV: /conte

X_train_seq shape: (241, 16, 224, 224, 3)
y_train_seq shape: (241,)
X_val_seq shape: (53, 16, 224, 224, 3)
y_val_seq shape: (53,)
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 16, 224, 224, 3)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 16, 7, 7, 1280)      │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 16, 1280)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 128)                 │         721,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,771,108 (18.20 MB)

 Trainable params: 721,537 (2.75 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

Epoch 1/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 665s 13s/step - accuracy: 0.6839 - loss: 0.5584 - val_accuracy: 1.0000 - val_loss: 0.1033 - learning_rate: 1.0000e-04
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 405s 12s/step - accuracy: 1.0000 - loss: 0.0587 - val_accuracy: 1.0000 - val_loss: 0.0065 - learning_rate: 1.0000e-04
Epoch 3/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 367s 12s/step - accuracy: 1.0000 - loss: 0.0051 - val_accuracy: 1.0000 - val_loss: 0.0022 - learning_rate: 1.0000e-04
Epoch 4/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 380s 12s/step - accuracy: 1.0000 - loss: 0.0020 - val_accuracy: 1.0000 - val_loss: 0.0012 - learning_rate: 1.0000e-04
Epoch 5/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 378s 12s/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 1.0000 - val_loss: 8.1002e-04 - learning_rate: 1.0000e-04
Epoch 6/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 328s 11s/step - accuracy: 1.0000 - loss: 9.9080e-04 - val_accuracy: 1.0000 - val_loss: 5.8060e-04 - learning_rate: 1.0000e-04
Epoch 7/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 314s 10s/step 